In [ ]:
#!pip install gensim==3.6.0

In [ ]:
import gensim
gensim.__version__

'3.6.0'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir("/content/drive/MyDrive/Datasets")

In [ ]:
import numpy as np
import re
import nltk
import pandas as pd
from nltk.corpus import stopwords
from gensim.models import  Word2Vec
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import roc_auc_score

In [ ]:
dataset = pd.read_csv('SPAM.csv')

In [ ]:
dataset.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
################### Replacing Apostrophe to Word #################
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    phrase = re.sub(r"don\'t", "do not", phrase)
    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase
###################################################################

In [ ]:
y = np.where(dataset['Category']=="spam",1,0)

In [ ]:
y

array([0, 0, 1, ..., 0, 0, 0])

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
stops = stopwords.words('english')

In [ ]:
corpus = []
for i in range(0, dataset.shape[0]):
    review = dataset['Message'][i]
    review = decontracted(review)
    review = re.sub('[^a-zA-Z]', ' ', review)
    review = review.lower()
#    ps = PorterStemmer()
    review = review.split()
#    review = [word for word in review if not word in set(stops)]
#    review = ' '.join(review)
#    review = [review]
    if review == []:
      review = ["anything"]
    corpus.append(review)

In [ ]:
len(corpus)

5572

In [ ]:
len(corpus)

5572

CBOW: sg = 0
Skip-Gram: sg = 1

In [ ]:
model_r =  Word2Vec(corpus, min_count=1,sg=0)

In [ ]:
means = []
for i in corpus :
    if i != []:
      row_means = np.mean(model_r[i],axis=0)
      means.append(row_means)
means = np.array(means)

X = means


<ipython-input-18-9e1505a289ed>:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  row_means = np.mean(model_r[i],axis=0)


In [ ]:
X.shape

(5572, 100)

In [ ]:
X.shape

(5572, 100)

In [ ]:
y.shape

(5572,)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, accuracy_score


X_train, X_test, y_train, y_test = train_test_split(X, y,test_size = 0.3,
                                                    random_state=2022,
                                                    stratify=y)

from xgboost import XGBClassifier
model_rf = XGBClassifier(random_state=2022,oob_score=True)
model_rf.fit( X_train , y_train )

XGBClassifier(oob_score=True, random_state=2022)

**Model Evaluation**


In [ ]:
y_pred_prob = model_rf.predict_proba(X_test)[:,1]

print(roc_auc_score(y_test, y_pred_prob))

0.9744968429360694


#### Grid Search CV

In [ ]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=2022)
model_xgb = XGBClassifier(random_state=2022)
print(model_xgb.get_params())

{'base_score': 0.5, 'booster': 'gbtree', 'colsample_bylevel': 1, 'colsample_bynode': 1, 'colsample_bytree': 1, 'gamma': 0, 'learning_rate': 0.1, 'max_delta_step': 0, 'max_depth': 3, 'min_child_weight': 1, 'missing': None, 'n_estimators': 100, 'n_jobs': 1, 'nthread': None, 'objective': 'binary:logistic', 'random_state': 2022, 'reg_alpha': 0, 'reg_lambda': 1, 'scale_pos_weight': 1, 'seed': None, 'silent': None, 'subsample': 1, 'verbosity': 1}


In [ ]:
params = {'learning_rate':[0.01, 0.1, 0.2, 0.5],
          'max_depth':[3, 6], 'n_estimators':[50, 100]}
gcv = GridSearchCV(model_xgb, param_grid=params, cv=kfold, scoring='roc_auc', verbose=3)
gcv.fit(X, y)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV 1/5] END learning_rate=0.01, max_depth=3, n_estimators=50;, score=0.953 total time=   1.4s
[CV 2/5] END learning_rate=0.01, max_depth=3, n_estimators=50;, score=0.944 total time=   1.3s
[CV 3/5] END learning_rate=0.01, max_depth=3, n_estimators=50;, score=0.971 total time=   1.4s
[CV 4/5] END learning_rate=0.01, max_depth=3, n_estimators=50;, score=0.940 total time=   1.3s
[CV 5/5] END learning_rate=0.01, max_depth=3, n_estimators=50;, score=0.972 total time=   1.3s
[CV 1/5] END learning_rate=0.01, max_depth=3, n_estimators=100;, score=0.962 total time=   2.6s
[CV 2/5] END learning_rate=0.01, max_depth=3, n_estimators=100;, score=0.945 total time=   2.6s
[CV 3/5] END learning_rate=0.01, max_depth=3, n_estimators=100;, score=0.973 total time=   2.6s
[CV 4/5] END learning_rate=0.01, max_depth=3, n_estimators=100;, score=0.943 total time=   2.6s
[CV 5/5] END learning_rate=0.01, max_depth=3, n_estimators=100;, score=0.975 tot

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=2022, shuffle=True),
             estimator=XGBClassifier(random_state=2022),
             param_grid={'learning_rate': [0.01, 0.1, 0.2, 0.5],
                         'max_depth': [3, 6], 'n_estimators': [50, 100]},
             scoring='roc_auc', verbose=3)

In [ ]:
print(gcv.best_params_)

{'learning_rate': 0.2, 'max_depth': 3, 'n_estimators': 100}


In [ ]:
print(gcv.best_score_)

0.9772476127551553
